In [ ]:
import os, sys, warnings
import pandas as pd
import numpy as np
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup

In [ ]:
# get links to all extractant pages
url = f'https://www.oecd-nea.org/ideal/'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10000)
soup = BeautifulSoup(response.text, 'html.parser')
t = soup.find_all('p')

ligand_links = []
base_url = f'https://www.oecd-nea.org'
for tt in t:
	all_a = tt.find_all('a')
	for a in all_a:
		if 'ideal' in a['href']:
			ligand_links.append(base_url + a['href'])


In [ ]:
table_mapping = {
	"Separation factors": "SF",
	"Distribution coefficients": "D",
}

def split_acid(col_name):
	if isinstance(col_name, str):  # Check if it's a string (not NaN)
		parts = col_name.split()
		if len(parts) >= 2: # Check if the split produced at least 2 elements
			return parts[0], float(parts[1].replace('M','')) # Return acid type and concentration
		else:
			return None, None #Handle cases where the split doesn't produce enough elements
	return None, None  # Handle NaN or other non-string values


df_D_cols = ['sample', 'D', 'extractant', 'abbreviation', '[extracant]_M', 'org_ligand', '[org_ligand]_M', 'organic_diluent', 'acid', '[acid]_M', 'T_K', 'contact_time', 'contact_time_units', 'additional_info']
df_SF_cols = ['sample', 'SF', 'extractant', 'abbreviation', '[extracant]_M', 'org_ligand', '[org_ligand]_M', 'organic_diluent', 'acid', '[acid]_M', 'T_K', 'contact_time', 'contact_time_units', 'additional_info']

df_D = pd.DataFrame(columns=df_D_cols)
df_SF = pd.DataFrame(columns=df_SF_cols)

for url in ligand_links:
	# read in url
	response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10000)
	soup = BeautifulSoup(response.text, 'html.parser')

	# get extractant name
	extractant = soup.find('h1').text
	print(extractant)

	abbrev = soup.find('p').text.replace('\n', '').replace(' ','').strip().split(',')[0]

	# get all tables
	tables = soup.find_all('table')
	for table in tables:
		table_title = table.find_previous('h5').text
		
		# check if table is in mapping
		if table_title in table_mapping.keys():
			# get data from table
			df = pd.read_html(str(table))[0]

			# clean up data
			df['Value'] = df['Value'].astype(str).str.extract(r'([0-9.]+)').astype(float)
			df['extractant'] = extractant
			df['abbreviation'] = abbrev
			
			try:
				df['[extracant]_M'] = df['Conc.'].str.extract(r'([0-9.]+)').astype(float)
			except:
				continue
				
			try:
				df[['acid', '[acid]_M']] = df['Aq. acid'].apply(split_acid).tolist()
			except:
				df['acid'] = np.nan
				df['[acid]_M'] = np.nan	
			
			try:
				df[['org_ligand', '[org_ligand]_M']] = df['Add. ligand'].apply(split_acid).tolist()
			except:
				df['org_ligand'] = np.nan
				df['[org_ligand]_M'] = np.nan

			try:
				df['T_K'] = df['Temp.'].str.extract(r'([0-9.]+)').astype(float)
				df['T_units'] = df['Temp.'].str.extract(r'([A-Za-z]+)')
				# convert to Kelvin
				C_loc = df[df['T_units'] == 'C'].index.tolist()
				df.loc[C_loc, 'T_K'] = df.loc[C_loc, 'T_K'] + 273.15
			except:
				df['T_K'] = np.nan

			try:
				df[['contact_time', 'contact_time_units']] = df['Contact time'].str.extract(r'([0-9.]+)\s*([A-Za-z]+)')
			except:
				df['contact_time'] = np.nan
				df['contact_time_units'] = np.nan

			# drop necessary columns
			df.drop(columns=['Aq. acid', 'Add. ligand', 'Temp.', 'Conc.', 'Ref.', 'Contact time'], inplace=True)
			# rename columns
			df.rename(columns={'Organic diluent': 'organic_diluent', 'Additional': 'additional_info'}, inplace=True)

			# finish data cleanup by dataframe type
			# add to correct dataframe
			if table_title == "Distribution coefficients":
				df.rename(columns={'Element': 'sample', 'Value': 'D'}, inplace=True)
				df = df[df_D_cols]
				df_D = df_D._append(df, ignore_index=True)
			elif table_title == "Separation factors":
				df.rename(columns={'Factor': 'sample', 'Value': 'SF'}, inplace=True)
				df = df[df_SF_cols]
				df_SF = df_SF._append(df, ignore_index=True)


In [ ]:
df_D.to_csv(f"D.csv", index=False)
df_SF.to_csv(f"SF.csv", index=False)
